# **Let's fine-tune cosmosGPT on ... dataset**

CosmosGPT is a GPT-2 based 774 million parameter model, so it'll be quite fast. (quite fast... hehe)

To show how fine-tuning affects a model, same instruction will be answered before and after fine-tuning.

In [1]:
!pip install transformers
!pip install accelerate


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import AutoTokenizer, GPT2LMHeadModel
from transformers import pipeline

c:\Users\yusuf\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\yusuf\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\yusuf\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
#model = GPT2LMHeadModel.from_pretrained("ytu-ce-cosmos/turkish-gpt2")
#tokenizer = AutoTokenizer.from_pretrained("ytu-ce-cosmos/turkish-gpt2")

# Use these lines to pull the model from Huggingface.

In [4]:
#model.save_pretrained("MYcosmosGPT")
#tokenizer.save_pretrained("MYcosmosGPT")

# Use these lines to save the loaded model locally.

In [5]:
model = GPT2LMHeadModel.from_pretrained("./MycosmosGPT")
tokenizer = AutoTokenizer.from_pretrained("./MycosmosGPT")

# Use these lines to load a locally saved model.

In [6]:
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.pad_token_id = tokenizer.eos_token_id

In [7]:
#Workflow of generating: Choose prompt, encode it, feed it to the model, decode the output.

prompt = "Zor günlere rağmen hayatı sevmek için 5 neden sırala."

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

output = model.generate(
    input_ids=input_ids,
    pad_token_id=tokenizer.pad_token_id,
    max_length=100
)

output

tensor([[18486, 23877,  2106,  5677, 29186,   373,   743,   843,   915,  3624,
            14,   603, 50174,   277,  7993,   717,   199, 18486, 23877,  2106,
          5677, 29186,   373,   743,   843,   915,  3624,    14,   199, 18486,
          6310, 22447,    14,   517,  1010,  8297, 11112,   417,   373,  3064,
         32522,    31,   199, 18486,  6310, 22447,    14,   517,  1010,  8297,
         11112,   417,   373,  3064, 32522,    31,   199, 18486,  6310, 22447,
            14,   517,  1010,  8297, 11112,   417,   373,  3064, 32522,    31,
           199, 18486,  6310, 22447,    14,   517,  1010,  8297, 11112,   417,
           373,  3064, 32522,    31,   199, 18486,  6310, 22447,    14,   517,
          1010,  8297, 11112,   417,   373,  3064, 32522,    31,   199, 18486]])

In [8]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [9]:
text = tokenizer.decode(output[0], skip_special_tokens=False)
print(text)

Zor günlere rağmen hayatı sevmek için 5 neden sırala. - KizlarSoruyor
Zor günlere rağmen hayatı sevmek için 5 neden sırala.
Zor günler geçirdik. Bu zor günleri atlatmak için neler yapmalıyız?
Zor günler geçirdik. Bu zor günleri atlatmak için neler yapmalıyız?
Zor günler geçirdik. Bu zor günleri atlatmak için neler yapmalıyız?
Zor günler geçirdik. Bu zor günleri atlatmak için neler yapmalıyız?
Zor günler geçirdik. Bu zor günleri atlatmak için neler yapmalıyız?
Zor


## Now let's fine-tune the model and generate outputs for the same instruction.

In [10]:
!pip uninstall huggingace_hub
!pip install huggingface_hub

!pip install datasets


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import datasets
from datasets import load_dataset
from datasets import Dataset as Ds
from torch.utils.data import Dataset
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [12]:
ds_len = 2000       #You can experiment with less training to see the result.
                    #We'll just use a tiny bit of the dataset.

dataset = load_dataset("merve/turkish_instructions")["train"].select(range(ds_len))

In [13]:
dataset

Dataset({
    features: ['Unnamed: 0', 'talimat', ' giriş', ' çıktı'],
    num_rows: 1000
})

In [14]:
train_dataset = { "example":  [talimat + çıktı for talimat, çıktı in zip(dataset["talimat"], dataset[" çıktı"])]} 
train_dataset = Ds.from_dict(train_dataset)

new_dataset = datasets.DatasetDict( {"train":train_dataset} )
new_dataset = new_dataset["train"]

# GPT-2 gets continous text as input, rather than separate instruction-answer pairs. So we concatanate them. Makes training longer.

In [15]:
new_dataset

Dataset({
    features: ['example'],
    num_rows: 1000
})

In [16]:
class MyDataSet(Dataset):
  def __init__(self, tokenizer, data, block_size):
    self.tokenizer = tokenizer
    self.data = data
    self.block_size = block_size

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    item = self.data[idx]
    tokenized_inputs = self.tokenizer(
        item["example"],
        truncation=True,
        padding="max_length",
        max_length=self.block_size,
        return_tensors="pt"
    )
    return tokenized_inputs

In [17]:
data = MyDataSet(tokenizer, new_dataset, 512)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
print(dir(data))

['__add__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_is_protocol', 'block_size', 'data', 'tokenizer']


In [18]:
print(data.data)
print(data.tokenizer)

Dataset({
    features: ['example'],
    num_rows: 1000
})
GPT2TokenizerFast(name_or_path='./MycosmosGPT', vocab_size=50257, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}


In [19]:
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    num_train_epochs=10,
    learning_rate=5e-5,
    logging_steps=100,
    output_dir='./results'
)

trainer= Trainer(
    model=model,
    args=training_args,
    train_dataset=data,
    eval_dataset=None,
    data_collator=data_collator,
)

c:\Users\yusuf\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [20]:
# These lines are used to monitor and optimize GPU memory usage.

#torch.cuda.memory_allocated()

#import os

#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
#torch.cuda.empty_cache()

#!nvidia-smi

In [21]:
trainer.train()

  0%|          | 0/1250 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
prompt ="Zor günlere rağmen hayatı sevmek için 5 neden sırala."

input_ids = tokenizer(prompt, return_tensors ="pt").input_ids
attention_mask = tokenizer(prompt, return_tensors = "pt").attention_mask

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

input_ids

tensor([[18486, 23877,  2106,  5677, 29186,   373,   743,   843,   915,  3624,
            14]], device='cuda:0')

In [ ]:
output = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    pad_token_id=tokenizer.pad_token_id,
    max_length=100,
    num_beams=5,
    temperature=1.5,
    top_k=50,
    do_sample=True
)

In [ ]:
output

tensor([[18486, 23877,  2106,  5677, 29186,   373,   743,   843,   915,  3624,
            14,    17,    14,  7191,  3937,   299, 21882,   439,    26,  7191,
          3937,   299, 21882,  4106,  5083,    14,   386,    14, 17111,    26,
         17111,   373,   307,  2202,    12,   307,  2202,   299,   307,  2202,
           483,    14,   609,    14, 37075,   439,    26,   790,  4137,  1910,
           307,   838, 22503, 14603,    14,   750,    14, 43812,  3731,    26,
           790,  4137,  3628,   307,  1824,  1045,   721,   307,  4552, 14422,
          6538, 24994,    14,   743,    14,  7507,  2985,    26,   790,  4137,
          3628,   307,  1824,  1045,   721,   307,  4552, 14422, 19025, 24994,
            14,   950,    14, 21401,   808,    26,   790,  4137,  3628,   307]],
       device='cuda:0')

In [ ]:
text = tokenizer.decode(output[0], skip_special_tokens=True)
print(text)

Zor günlere rağmen hayatı sevmek için 5 neden sırala.1. Tutku ve iyimserlik: Tutku ve iyimserliğin gücü. 2. Başarı: Başarı için bir amaç, bir amaç ve bir amaçtır. 3. Esneklik: Bir sorunu hızlı bir şekilde çözme yeteneği. 4. Yaratıcılık: Bir sorunu belirli bir süre içinde yeni bir bakış açısı kazanma becerisi. 5. Kararlılık: Bir sorunu belirli bir süre içinde yeni bir bakış açısı getirme becerisi. 6. Cesaret: Bir sorunu belirli bir


In [ ]:
trainer.save_model("SFT-cosmos-gpt2-merve-2k")

In [ ]:
tokenizer.save_pretrained("SFT-cosmos-gpt2-merve-2k")

('SFT-cosmos-gpt2-merve8k-33p\\tokenizer_config.json',
 'SFT-cosmos-gpt2-merve8k-33p\\special_tokens_map.json',
 'SFT-cosmos-gpt2-merve8k-33p\\vocab.json',
 'SFT-cosmos-gpt2-merve8k-33p\\merges.txt',
 'SFT-cosmos-gpt2-merve8k-33p\\added_tokens.json',
 'SFT-cosmos-gpt2-merve8k-33p\\tokenizer.json')